# Download the data files

In [91]:
interactions_filename = 'INTERACTIONS.csv'
users_filename = 'USERS.csv'

In [92]:
!wget https://tempbloggingbucket.s3-ap-southeast-1.amazonaws.com/customer11k.csv  -O USERS.csv
!wget https://tempbloggingbucket.s3-ap-southeast-1.amazonaws.com/interactions.csv -O INTERACTIONS.csv

--2020-09-27 15:10:54--  https://tempbloggingbucket.s3-ap-southeast-1.amazonaws.com/customer11k.csv
Resolving tempbloggingbucket.s3-ap-southeast-1.amazonaws.com (tempbloggingbucket.s3-ap-southeast-1.amazonaws.com)... 52.219.133.55
Connecting to tempbloggingbucket.s3-ap-southeast-1.amazonaws.com (tempbloggingbucket.s3-ap-southeast-1.amazonaws.com)|52.219.133.55|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 456961 (446K) [text/csv]
Saving to: ‘USERS.csv’

USERS.csv           100%[===================>] 446.25K  --.-KB/s    in 0.01s   

2020-09-27 15:10:55 (38.5 MB/s) - ‘USERS.csv’ saved [456961/456961]

--2020-09-27 15:10:55--  https://tempbloggingbucket.s3-ap-southeast-1.amazonaws.com/interactions.csv
Resolving tempbloggingbucket.s3-ap-southeast-1.amazonaws.com (tempbloggingbucket.s3-ap-southeast-1.amazonaws.com)... 52.219.133.55
Connecting to tempbloggingbucket.s3-ap-southeast-1.amazonaws.com (tempbloggingbucket.s3-ap-southeast-1.amazonaws.com)|52.219.133.55|

# Organize Python imports

In [110]:
import boto3
import json
import numpy as np
import pandas as pd
import csv
import string 
import time


# Check the data 

In [94]:
users_df=pd.read_csv("USERS.csv")
interactions_df=pd.read_csv("INTERACTIONS.csv")

In [95]:
print(users_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11162 entries, 0 to 11161
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   USER_ID    11162 non-null  int64 
 1   deposit    11162 non-null  object
 2   age        11162 non-null  int64 
 3   job        11162 non-null  object
 4   marital    11162 non-null  object
 5   education  11162 non-null  object
dtypes: int64(2), object(4)
memory usage: 523.3+ KB
None


In [96]:
print(interactions_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32220 entries, 0 to 32219
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   USER_ID    32220 non-null  int64 
 1   ITEM_ID    32220 non-null  object
 2   TIMESTAMP  32220 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 755.3+ KB
None


In [97]:
users_df.head()

,USER_ID,deposit,age,job,marital,education
0,30908,yes,59,admin.,married,secondary
1,30909,yes,56,admin.,married,secondary
2,30910,yes,41,technician,married,secondary
3,30911,yes,55,services,married,secondary
4,30912,yes,54,admin.,married,tertiary


In [98]:
interactions_df.head()

,USER_ID,ITEM_ID,TIMESTAMP
0,35739,AC_NE,1579609263
1,35739,AC_AS,1580612023
2,35739,AC_PA,1581390495
3,35739,AC_AA,1582229479
4,38695,TA_NE,1575001446


# Upload data to S3

In [84]:
random_suffix = ''.join(np.random.choice(list(string.ascii_lowercase + string.digits), 5))
print(random_suffix)

zhok6


In [121]:
bucket = "personalize-connect-blog-data-"+random_suffix     # replace with the name of your S3 bucket
!aws s3 mb s3://{bucket}

In [89]:
personalize = boto3.client(service_name='personalize')
personalize_runtime = boto3.client(service_name='personalize-runtime')

In [100]:
boto3.Session().resource('s3').Bucket(bucket).Object(interactions_filename).upload_file(interactions_filename)

In [101]:
boto3.Session().resource('s3').Bucket(bucket).Object(users_filename).upload_file(users_filename)

# Create Personalize Service Artifacts using boto3 APIs

## Create Personalize schemas

In [105]:
users_schema_name='personalize-connect-blog-users-schema-'+random_suffix
interactions_schema_name='personalize-connect-blog-interaction-schema-'+random_suffix
print(users_schema_name)

personalize-connect-blog-users-schema-zhok6


In [106]:
users_schema = {
    "type": "record",
    "name": "Users",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "USER_ID",
            "type": "string"
        },
        {
            "name": "deposit",
            "type": "string"
        },
        {
            "name": "age",
            "type": "long"
        },
        {
            "name": "job",
            "type": "string"
        },
        {
            "name": "marital",
            "type": "string"
        },
        {
            "name": "education",
            "type": "string"
        }
    ],
    "version": "1.0"
}

create_schema_response = personalize.create_schema(
    name = users_schema_name,
    schema = json.dumps(users_schema)
)

users_schema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))

{
  "schemaArn": "arn:aws:personalize:ap-southeast-1:248025046818:schema/personalize-connect-blog-users-schema-zhok6",
  "ResponseMetadata": {
    "RequestId": "04148010-4181-448c-8812-3eb731972f45",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Sun, 27 Sep 2020 15:20:56 GMT",
      "x-amzn-requestid": "04148010-4181-448c-8812-3eb731972f45",
      "content-length": "114",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [107]:
interactions_schema =  {
  "type": "record",
  "name": "Interactions",
  "namespace": "com.amazonaws.personalize.schema",
  "fields": [
      {
          "name": "USER_ID",
          "type": "string"
      },
      {
          "name": "ITEM_ID",
          "type": "string"
      },
      {
          "name": "TIMESTAMP",
          "type": "long"
      }
  ],
  "version": "1.0"
}

create_schema_response = personalize.create_schema(
    name = interactions_schema_name,
    schema = json.dumps(interactions_schema)
)

interactions_schema_arn = create_schema_response['schemaArn']

print(json.dumps(create_schema_response, indent=2))

{
  "schemaArn": "arn:aws:personalize:ap-southeast-1:248025046818:schema/personalize-connect-blog-interaction-schema-zhok6",
  "ResponseMetadata": {
    "RequestId": "f55aa611-664e-422a-bea9-c4a3fea1ec96",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Sun, 27 Sep 2020 15:22:01 GMT",
      "x-amzn-requestid": "f55aa611-664e-422a-bea9-c4a3fea1ec96",
      "content-length": "120",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


## Create Dataset Group and Datasets

In [108]:
create_dataset_group_response = personalize.create_dataset_group(
    name = "personalize-connect-blog-dataset-"+random_suffix
)

dataset_group_arn = create_dataset_group_response['datasetGroupArn']
print(json.dumps(create_dataset_group_response, indent=2))

{
  "datasetGroupArn": "arn:aws:personalize:ap-southeast-1:248025046818:dataset-group/personalize-connect-blog-dataset-zhok6",
  "ResponseMetadata": {
    "RequestId": "66b82a8b-e495-4ad3-af4a-3269d8607a8a",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Sun, 27 Sep 2020 15:25:01 GMT",
      "x-amzn-requestid": "66b82a8b-e495-4ad3-af4a-3269d8607a8a",
      "content-length": "122",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [111]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_group_response = personalize.describe_dataset_group(
        datasetGroupArn = dataset_group_arn
    )
    status = describe_dataset_group_response["datasetGroup"]["status"]
    print("DatasetGroup: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

DatasetGroup: ACTIVE


In [112]:
dataset_type = "INTERACTIONS"
create_dataset_response = personalize.create_dataset(
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = interactions_schema_arn,
    name = "personalize-connect-blog-interaction-dataset-"+random_suffix
)

interactions_dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))


{
  "datasetArn": "arn:aws:personalize:ap-southeast-1:248025046818:dataset/personalize-connect-blog-dataset-zhok6/INTERACTIONS",
  "ResponseMetadata": {
    "RequestId": "d3e1c360-4389-4429-a3b7-5cee5cc2e077",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Sun, 27 Sep 2020 15:30:04 GMT",
      "x-amzn-requestid": "d3e1c360-4389-4429-a3b7-5cee5cc2e077",
      "content-length": "124",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [115]:
dataset_type = "USERS"
create_dataset_response = personalize.create_dataset(
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = users_schema_arn,
    name = "personalize-connect-blog-user-dataset-"+random_suffix
)

users_dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))


{
  "datasetArn": "arn:aws:personalize:ap-southeast-1:248025046818:dataset/personalize-connect-blog-dataset-zhok6/USERS",
  "ResponseMetadata": {
    "RequestId": "ac65202b-0356-4e60-a95a-8d9f9e57b128",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Sun, 27 Sep 2020 15:32:41 GMT",
      "x-amzn-requestid": "ac65202b-0356-4e60-a95a-8d9f9e57b128",
      "content-length": "117",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


## Add S3 bucket policy for accessing from personalize

In [128]:
s3 = boto3.client("s3")

policy = {
    "Version": "2012-10-17",
    "Id": "PersonalizeS3BucketAccessPolicy",
    "Statement": [
        {
            "Sid": "PersonalizeS3BucketAccessPolicy",
            "Effect": "Allow",
            "Principal": {
                "Service": "personalize.amazonaws.com"
            },
            "Action": [
                "s3:GetObject",
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::{}".format(bucket),
                "arn:aws:s3:::{}/*".format(bucket)
            ]
        }
    ]
}

s3.put_bucket_policy(Bucket=bucket, Policy=json.dumps(policy));

## Use the Personalize IAM Role 

In [118]:
role_arn = 'arn:aws:iam::248025046818:role/AmazonPersonalizeRole'  ## Replace with your IAM role name for personalize

## Create Data Import Role

In [122]:
print(bucket)

personalize-connect-blog-data-zhok6


In [129]:
create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "interactions-dataset-import-"+random_suffix,
    datasetArn = interactions_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket, interactions_filename)
        
    }, roleArn = role_arn
)

interactions_dataset_import_job_arn = create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_dataset_import_job_response, indent=2))

{
  "datasetImportJobArn": "arn:aws:personalize:ap-southeast-1:248025046818:dataset-import-job/interactions-dataset-import-zhok6",
  "ResponseMetadata": {
    "RequestId": "3f291df3-04f7-4b8b-87c5-27484c3ecfac",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Sun, 27 Sep 2020 15:47:21 GMT",
      "x-amzn-requestid": "3f291df3-04f7-4b8b-87c5-27484c3ecfac",
      "content-length": "126",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [130]:
create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "users-dataset-import-"+random_suffix,
    datasetArn = users_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket, users_filename)
        
    }, roleArn = role_arn
)

users_dataset_import_job_arn = create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_dataset_import_job_response, indent=2))

{
  "datasetImportJobArn": "arn:aws:personalize:ap-southeast-1:248025046818:dataset-import-job/users-dataset-import-zhok6",
  "ResponseMetadata": {
    "RequestId": "fe919d60-1845-429b-b9a2-4b96989d3a09",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Sun, 27 Sep 2020 15:47:24 GMT",
      "x-amzn-requestid": "fe919d60-1845-429b-b9a2-4b96989d3a09",
      "content-length": "119",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [131]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = users_dataset_import_job_arn
    )
    
    dataset_import_job = describe_dataset_import_job_response["datasetImportJob"]
    if "latestDatasetImportJobRun" not in dataset_import_job:
        status = dataset_import_job["status"]
        print("DatasetImportJob: {}".format(status))
    else:
        status = dataset_import_job["latestDatasetImportJobRun"]["status"]
        print("LatestDatasetImportJobRun: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

DatasetImportJob: CREATE PENDING
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: ACTIVE


In [132]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = interactions_dataset_import_job_arn
    )
    
    dataset_import_job = describe_dataset_import_job_response["datasetImportJob"]
#     print(dataset_import_job)
    if "latestDatasetImportJobRun" not in dataset_import_job:
        status = dataset_import_job["status"]
        print("DatasetImportJob: {}".format(status))
    else:
        status = dataset_import_job["latestDatasetImportJobRun"]["status"]
        print("LatestDatasetImportJobRun: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: ACTIVE


In [133]:
recipe_list = personalize.list_recipes()
for recipe in recipe_list['recipes']:
    print(recipe['recipeArn'])

arn:aws:personalize:::recipe/aws-hrnn
arn:aws:personalize:::recipe/aws-hrnn-coldstart
arn:aws:personalize:::recipe/aws-hrnn-metadata
arn:aws:personalize:::recipe/aws-personalized-ranking
arn:aws:personalize:::recipe/aws-popularity-count
arn:aws:personalize:::recipe/aws-sims
arn:aws:personalize:::recipe/aws-user-personalization


In [134]:
recipe_arn = "arn:aws:personalize:::recipe/aws-hrnn"

## Create a Solution Defition in Personalize Service

In [135]:
create_solution_response = personalize.create_solution(
    name = 'demo-hrnn-solution-'+ random_suffix ,
    datasetGroupArn = dataset_group_arn,
    recipeArn = recipe_arn
)

solution_arn = create_solution_response['solutionArn']
print(json.dumps(create_solution_response, indent=2))

{
  "solutionArn": "arn:aws:personalize:ap-southeast-1:248025046818:solution/demo-hrnn-solution-zhok6",
  "ResponseMetadata": {
    "RequestId": "2f586d96-a43a-4d6f-8302-129ea7ec4171",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Sun, 27 Sep 2020 22:34:35 GMT",
      "x-amzn-requestid": "2f586d96-a43a-4d6f-8302-129ea7ec4171",
      "content-length": "99",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [136]:
create_solution_version_response = personalize.create_solution_version(
    solutionArn = solution_arn
)

solution_version_arn = create_solution_version_response['solutionVersionArn']
print(json.dumps(create_solution_version_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:ap-southeast-1:248025046818:solution/demo-hrnn-solution-zhok6/c2791742",
  "ResponseMetadata": {
    "RequestId": "ff0e2d82-a3e1-4771-9915-cf354e4dc7e4",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Sun, 27 Sep 2020 22:35:25 GMT",
      "x-amzn-requestid": "ff0e2d82-a3e1-4771-9915-cf354e4dc7e4",
      "content-length": "115",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [ ]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_solution_version_response = personalize.describe_solution_version(
        solutionVersionArn = solution_version_arn
    )
    status = describe_solution_version_response["solutionVersion"]["status"]
    print("SolutionVersion: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_P

## Deploy & Create Campaign URL

In [139]:
create_campaign_response = personalize.create_campaign(
    name = "demo-hrnn-campaign-"+random_suffix,
    solutionVersionArn = solution_version_arn,
    minProvisionedTPS = 2,  
)

campaign_arn = create_campaign_response['campaignArn']
print(json.dumps(create_campaign_response, indent=2))

{
  "campaignArn": "arn:aws:personalize:ap-southeast-1:248025046818:campaign/demo-hrnn-campaign-zhok6",
  "ResponseMetadata": {
    "RequestId": "11899171-7f5e-4352-98d4-3e7d07abc255",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Sun, 27 Sep 2020 23:48:36 GMT",
      "x-amzn-requestid": "11899171-7f5e-4352-98d4-3e7d07abc255",
      "content-length": "99",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [140]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_campaign_response = personalize.describe_campaign(
        campaignArn = campaign_arn
    )
    status = describe_campaign_response["campaign"]["status"]
    print("Campaign: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

Campaign: CREATE PENDING
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: ACTIVE


# Get Customer Intent Predicted 

## Prediction for existing data

In [146]:
response = personalize_runtime.get_recommendations(
    campaignArn = campaign_arn,
    userId = '37604' )

# 35071
# 37604
# 36720
# 37003

print("Recommended items - score ")
for item in response['itemList']:
     print (item['itemId'], item['score'])

Recommended items - score 
TA_CS 0.9562176
TA_NE 0.0105909
HL_NE 0.0070264
AC_NE 0.0063722
FD_NE 0.0059742
CC_NE 0.0049404
LI_NE 0.0043251
HL_CS 0.0010232
LI_CS 0.000626
LI_PA 0.0005488
HL_RP 0.0003806
HL_AS 0.0002819
CC_AA 0.0002276
FD_PW 0.0001629
CC_CS 0.0001492
CC_CA 0.000139
HL_DS 0.0001198
FD_CS 0.000117
FD_NA 0.0001092
AC_AA 0.0001012
FD_AD 9.96e-05
LI_AS 9.89e-05
CC_LC 8.29e-05
AC_AS 7.62e-05


## Prediction for new data

In [147]:
event_tracker = 'CallInentTracker'
response = personalize.create_event_tracker(
    name=event_tracker,
    datasetGroupArn=dataset_group_arn
)

tracking_id = response['trackingId']
print(response)

{'eventTrackerArn': 'arn:aws:personalize:ap-southeast-1:248025046818:event-tracker/e9f31936', 'trackingId': '399f6fb0-739f-40b2-88ff-e14f23932a7f', 'ResponseMetadata': {'RequestId': 'd19fa3d9-12f9-4e1c-9099-e1b1be583d45', 'HTTPStatusCode': 200, 'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1', 'date': 'Mon, 28 Sep 2020 00:06:53 GMT', 'x-amzn-requestid': 'd19fa3d9-12f9-4e1c-9099-e1b1be583d45', 'content-length': '144', 'connection': 'keep-alive'}, 'RetryAttempts': 0}}


In [148]:
import random
import string

personalize_events = boto3.client(service_name='personalize-events')

letters = string.ascii_lowercase
session_id= ''.join(random.choice(letters) for i in range(10)) 

In [149]:
personalize_events.put_events(
    trackingId = tracking_id,
    userId= '33874',
    sessionId = session_id,
    eventList = [{
        'sentAt': int(time.time()-604800),
        'eventType': 'call',
        'properties': "{\"itemId\": \"CC_NE\"}"
        },{
        'sentAt': int(time.time()-404800),
        'eventType': 'call',
        'properties': "{\"itemId\": \"CC_CA\"}"
        },{
        'sentAt': int(time.time()-94800),
        'eventType': 'call',
        'properties': "{\"itemId\": \"CC_AA\"}"
        }]
)

{'ResponseMetadata': {'RequestId': 'e31622b3-e79b-49ab-8497-a950199bbd3a',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/json',
   'date': 'Mon, 28 Sep 2020 00:07:21 GMT',
   'x-amzn-requestid': 'e31622b3-e79b-49ab-8497-a950199bbd3a',
   'content-length': '0',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

In [150]:
response = personalize_runtime.get_recommendations(
    campaignArn = campaign_arn,
    userId = '33874' )

print("Recommended items - scores")

for item in response['itemList']:
     print (item['itemId'], item['score'])

Recommended items - scores
CC_LC 0.9463729
CC_AA 0.0125381
CC_NE 0.0069692
FD_NE 0.0054608
TA_NE 0.0050527
AC_NE 0.0041886
HL_NE 0.003945
LI_NE 0.0038023
FD_PW 0.0026203
CC_CS 0.0012843
HL_AS 0.0010713
LI_AS 0.0009202
FD_CS 0.0008002
FD_AD 0.0007574
CC_CA 0.0006649
AC_AA 0.000574
FD_NA 0.00057
LI_PA 0.0004066
AC_CS 0.0003915
AC_PA 0.0002972
HL_RP 0.0002798
LI_CS 0.000238
LI_SW 0.00021
HL_DS 0.0002029
HL_CS 0.000163
